In [1]:
import numpy as np
import sys

class Unbuffered(object):
   def __init__(self, stream):
       self.stream = stream
   def write(self, data):
       self.stream.write(data)
       self.stream.flush()
   def writelines(self, datas):
       self.stream.writelines(datas)
       self.stream.flush()
   def __getattr__(self, attr):
       return getattr(self.stream, attr)

sys.stdout = Unbuffered(sys.stdout)


#embedding the position 
def pos_embed(x):
    if x < -60:
        return 0
    if x >= -60 and x <= 60:
        return x+61
    if x > 60:
        return 122
#find the index of x in y, if x not in y, return -1
def find_index(x,y):
    flag = -1
    for i in range(len(y)):
        if x != y[i]:
            continue
        else:
            return i
    return flag

In [3]:
print('reading word embedding data...')
vec = []
word2id = {}
f = open('./origin_data/vec.txt')
f.readline()
while True:
    content = f.readline()
    if content == '':
        break
    content = content.strip().split()
    word2id[content[0]] = len(word2id)
    content = content[1:]
    content = [(float)(i) for i in content]
    vec.append(content)
f.close()
word2id['UNK'] = len(word2id)
word2id['BLANK'] = len(word2id)

dim = 50
vec.append(np.random.normal(size=dim,loc=0,scale=0.05))
vec.append(np.random.normal(size=dim,loc=0,scale=0.05))
vec = np.array(vec,dtype=np.float32)


print('reading relation to id')
relation2id = {}
f = open('./origin_data/relation2id.txt','r')
while True:
    content = f.readline()
    if content == '':
        break
    content = content.strip().split()
    relation2id[content[0]] = int(content[1])
f.close()

#length of sentence is 70
fixlen = 70
#max length of position embedding is 60 (-60~+60)
maxlen = 60

train_sen = {} #{entity pair:[[[label1-sentence 1],[label1-sentence 2]...],[[label2-sentence 1],[label2-sentence 2]...]}
train_ans = {} #{entity pair:[label1,label2,...]} the label is one-hot vector


reading word embedding data...
reading relation to id


In [94]:
print('reading test data ...')

test_sen = {} #{entity pair:[[sentence 1],[sentence 2]...]}
test_ans = {} #{entity pair:[labels,...]} the labels is N-hot vector (N is the number of multi-label)

f = open('./origin_data/test.txt','r')

counter = 0

while True:
    content = f.readline()
    
    if content == '':
        break

    content = content.strip().split()
    en1 = content[2]
    en2 = content[3]
    relation = 0
    if content[4] not in relation2id:
        relation = relation2id['NA']
    else:
        relation = relation2id[content[4]]
        
    tup = (en1,en2)    
    
    if tup not in test_sen:
        test_sen[tup]=[]
        y_id = relation
        label_tag = 0
        label = [0 for i in range(len(relation2id))]
        label[y_id] = 1
        test_ans[tup] = label
    else:
        y_id = relation
        test_ans[tup][y_id] = 1
    
    sentence = content[5:-1] # remove ending tag
    
    # entity positions
    en1pos = 0
    en2pos = 0

    # need improvement from the wiki bio dataset
    for i in range(len(sentence)):
        if sentence[i] == en1:
            en1pos = i
        if sentence[i] == en2:
            en2pos = i
    output = []

    for i in range(fixlen):
        word = word2id['BLANK']
        rel_e1 = pos_embed(i - en1pos)
        rel_e2 = pos_embed(i - en2pos)
        output.append([word,rel_e1,rel_e2])
    
#     print(output)
#     print(len(output))
    
    for i in range(min(fixlen,len(sentence))):
        word = 0
        if sentence[i] not in word2id:
            word = word2id['UNK']
        else:
            word = word2id[sentence[i]]
        output[i][0] = word

#     print('=====')
#     print(output[0][0])
#     print(output[1][0])
#     print(len(output))
#     print(len(output[0]))

    
    test_sen[tup].append(output)
    
#     print('=======')
#     print(test_sen)
    
#     if counter == 0:
#         break
#     else:
#         counter += 1

reading test data ...


In [114]:
len(relation2id)

91

In [97]:
test_x = []
test_y = []

print('organizing test data')
f = open('./data/test_q&a.txt','w')
temp=0

print(len(test_sen))
print(len(test_ans))

for i in test_sen:
    test_x.append(test_sen[i]) # sentence
    test_y.append(test_ans[i]) #
    tempstr = ''
    for j in range(len(test_ans[i])):
        if test_ans[i][j]!=0:
            tempstr = tempstr+str(j)+'\t'
    f.write(str(temp)+'\t'+i[0]+'\t'+i[1]+'\t'+tempstr+'\n')
    temp+=1
f.close()

test_x = np.array(test_x)
test_y = np.array(test_y)

organizing test data
13265
13265


In [101]:
for i in test_sen:
    print(i)
    break

('renato_campos', 'defender')


In [103]:
len(test_y)

13265

In [106]:
import numpy as np
test_y = np.load('./data/testall_y.npy')

In [107]:
len(test_y)

13265

In [138]:
eval_y = []
for i in test_y:
#     print(i[1:])
#     print(len(i[1:]))
    eval_y.append(i[1:])
    
print(len(eval_y))    
allans = np.reshape(eval_y,(-1))
# np.save('./data/allans.npy',allans)

13265


In [126]:
type(eval_y)

list

In [139]:
print(len(eval_y))    

13265


In [140]:
len(allans)

1193850

In [141]:
13265*90

1193850

In [2]:
a = [1,2,3]